# Diffusion Limited Aggregation 

#### Author : B. Militzer, University of California, Berkeley 
#### Date   : Sept. 26, 2018

#### Read "The Science of Fractal Images", Ed. Peitgen and Saupe, p. 37 (1988)


In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#note, this function expects a matrix A[ix,iy] 
#and then displays so that A[:,0] is the lowest row of pixels
def display(A):
    maxX = A.shape[0]
    maxY = A.shape[1]
    B = np.zeros((maxY, maxX))
    for ix in range(0,maxX):
        for iy in range(0,maxY):
            B[maxY-1-iy,ix] = A[ix,iy]

    %matplotlib qt
    #On a PC, use 'qt' instead
    #plt.rcParams['figure.figsize'] = [15, 15/maxX*maxY]
    plt.imshow(B); 
    plt.axis('off'); 
    plt.show()
    plt.draw()
    plt.pause(0.01)

In [3]:
nParticles = 100*1000
maxX = 100
maxY = 100

In [4]:
# Initialize matrix containing all 2D grid points A(x,y)
# 0 <= x < maxX
# 0 <= y < maxY
# A(x,y)=0 ... site is empty
# A(x,y)>0 ... site is filled
A = np.zeros((maxX, maxY))

# Introduce a sticky wall at the bottom 
# by filling the lowest row of pixels with particles
A[:,0] = 1
print(A.transpose())

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
A.shape

(100L, 100L)

In [6]:
#test the display routine
display(A)

In [7]:
# To save computer time, we want to inject the new particle not too far
# above growing aggregate. We inject at on a line 'yStart', which
# keeps being increased so that it is always 'yBuffer' lines above the
# highest structure
yBuffer = 5
yStart  = 1 + yBuffer

In [8]:
np.random.random()

0.017114845928710265

In [9]:
for i in range(0,nParticles):
    # Compute new starting point on the line y=yStart
    x  = np.random.randint(0,maxX)
    y  = yStart; #always start at upper limit

    while True:
        xOrg = x
        yOrg = y
        r = np.random.random(); # Random float:  0.0 <= r < 1.0
        #based on the value of 'r', move the particle
        #left, right, up, or down and change x and y accordingly
        if r > 0 and r <= .25: #up
            x = x+1
        if r > .25 and r <= .5: #down
            x = x-1
        if r > .5 and r <= .75: #left
            y = y-1
        if r > .75 and r <= 1: #right
            y = y+1
            
            
        
        #now apply periodic boundary conditions to 'x'
        if x < 0:
            x = maxX-1
        if x > maxX-1:
            x = 0
        
        if (A[x,y] == 1 or y>yStart): 
            x = xOrg
            y = yOrg
            continue; # if this site has been taken try moving in a different direction
        
        #determine the x coordionates of the left and right neighbors
        #store them in 'xm' and 'xp' and apply periodic boundary conditions again
        xp = x+1
        xm = x-1
        
        if xp > maxX-1:
            xp = 0
        if xm < 0:
            xm = maxX-1
            
        yp = y+1
        ym = y-1
 
        # Determine if any neighboring site is occupied
        # if that is the case, enter the following 'if' clause
        if A[xm,y] == 1 or A[xp,y] == 1 or A[x,ym] == 1 or A[x,yp] ==1: 
            A[x,y] = 1
            if (y+yBuffer>yStart and y+yBuffer<maxY): 
                yStart = y+yBuffer

            if (i%1000==0): 
                print('i=',i,' \tx=',x,'\ty=',y,' \tyStart=',yStart)

            nNewParticlesPerFrame = 1000 
#             if (i%nNewParticlesPerFrame==0): 
#                 display(A)
                
            break # particle was attached, break out of current loop and insert next one
            
    if (yStart+1==maxY): 
        print('Structures reached Y limit after only', i ,'particles')
        break

display(A)

('i=', 0, ' \tx=', 46, '\ty=', 1, ' \tyStart=', 6)
('i=', 1000, ' \tx=', 35, '\ty=', 55, ' \tyStart=', 64)
('Structures reached Y limit after only', 1541, 'particles')
